<a href="https://colab.research.google.com/github/RegNLP/ContextAware-Regulatory-GraphRAG-ObliQAMP/blob/main/3_Experiment_2_Value_of_Graph_Re_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ==============================================================================
# Experiment 2: Enriched Graph Re-ranking (Dual Champion Test)
#
# Purpose:
# 1. Use the top two champion retrievers identified in Experiment 1.
# 2. For each champion, systematically test multiple graph re-ranking strategies:
#    - Hyperparameter tuning of bonus values.
#    - Expanded bonus types (Sibling, Cited By).
#    - PageRank centrality bonus.
#    - Penalty for isolated passages.
# 3. Evaluate all combinations to find the optimal retriever and graph strategy.
# ==============================================================================

# --- Essential Installations ---
# !pip install -q sentence-transformers pytrec_eval rank_bm25 pandas networkx

import os
import json
import torch
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import pytrec_eval
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
import itertools

# --- Configuration ---
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/RIRAG-MultiPassage-NLLP/"
GRAPH_PATH = os.path.join(BASE_PATH, "graph.gpickle")
TEST_SET_PATH = os.path.join(BASE_PATH, "QADataset", "ObliQA_MultiPassage_test.json")
QREL_PATH = os.path.join(BASE_PATH, "qrels.trec")
EMBEDDINGS_FOLDER = os.path.join(BASE_PATH, "embeddings")
RESULTS_OUTPUT_PATH = os.path.join(BASE_PATH, "experiment_2_enriched_graph_results_dual_champion.csv")

# --- Champion Retriever Configurations (from Experiment 1) ---
CHAMPION_CONFIGS = [
    {
        "name": "e5-large-parent_child",
        "model_key": "e5-large",
        "model_path": "intfloat/e5-large-v2",
        "context_key": "parent_child"
    },
    {
        "name": "e5-large-full_neighborhood",
        "model_key": "e5-large",
        "model_path": "intfloat/e5-large-v2",
        "context_key": "full_neighborhood"
    }
]


# --- Enriched Experiment Hyperparameters ---
K_INITIAL = 100  # Number of initial candidates from hybrid retrieval
K_FINAL = 20     # Final number of results to evaluate

# Define ranges for hyperparameters to test
PARENT_BONUS_OPTIONS = [0.01, 0.02, 0.05]
CITES_BONUS_OPTIONS = [0.01, 0.02] # Bonus for citing another retrieved doc
CITED_BY_BONUS_OPTIONS = [0.02, 0.04] # Bonus for being cited by another retrieved doc
SIBLING_BONUS_OPTIONS = [0.005, 0.01]
PAGERANK_WEIGHT_OPTIONS = [0.0, 0.1, 0.2] # 0.0 effectively disables this bonus
ISOLATION_PENALTY_OPTIONS = [0.0, -0.01] # 0.0 effectively disables this penalty

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# --- Load Static Components ---
print("Loading static components...")
with open(GRAPH_PATH, "rb") as f:
    G = pickle.load(f)

# --- Pre-calculate PageRank ---
print("Pre-calculating PageRank for all graph nodes...")
pagerank_scores = nx.pagerank(G, alpha=0.85)
print("PageRank calculation complete.")

with open(TEST_SET_PATH, "r", encoding="utf-8") as f:
    test_data = json.load(f)
print(f"Loaded {len(test_data)} test questions.")

# Load QRELs
qrel = {}
with open(QREL_PATH, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        qid, _, uid, rel = parts[0], parts[1], " ".join(parts[2:-1]), int(parts[-1])
        qrel.setdefault(qid, {})[uid] = rel
print(f"Loaded QRELs for {len(qrel)} queries.")

# Prepare BM25
print("Preparing BM25 index...")
all_passage_uids = [n for n, d in G.nodes(data=True) if d.get("type") == "Passage"]
corpus_texts = [G.nodes[uid].get("text", "") for uid in all_passage_uids]
tokenized_corpus = [text.split() for text in corpus_texts]
bm25 = BM25Okapi(tokenized_corpus)
print("BM25 ready.")


# --- Helper Functions ---
def evaluate_run(run_dict, qrel_dict, metrics={"recall_10", "map_cut_10", "ndcg_cut_10"}):
    evaluator = pytrec_eval.RelevanceEvaluator(qrel_dict, metrics)
    results = evaluator.evaluate(run_dict)
    agg = {metric: pytrec_eval.compute_aggregated_measure(metric, [r.get(metric, 0.0) for r in results.values()]) for metric in metrics}
    return agg

def reciprocal_rank_fusion(ranked_lists, k=60):
    fused = {}
    for lst in ranked_lists:
        for rank, uid in enumerate(lst):
            fused[uid] = fused.get(uid, 0) + 1 / (k + rank + 1)
    return sorted(fused.items(), key=lambda x: x[1], reverse=True)

# --- Main Experiment Loop ---
all_results = []
# Create all combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(
    PARENT_BONUS_OPTIONS,
    CITES_BONUS_OPTIONS,
    CITED_BY_BONUS_OPTIONS,
    SIBLING_BONUS_OPTIONS,
    PAGERANK_WEIGHT_OPTIONS,
    ISOLATION_PENALTY_OPTIONS
))

# --- Loop over each champion configuration ---
for champion_config in CHAMPION_CONFIGS:
    champion_name = champion_config["name"]
    champion_model_key = champion_config["model_key"]
    champion_model_path = champion_config["model_path"]
    champion_context_key = champion_config["context_key"]

    print("\n" + "="*80)
    print(f"--- TESTING CHAMPION RETRIEVER: {champion_name} ---")
    print("="*80)

    # --- Load Champion Embeddings ---
    print(f"Loading champion embeddings: {champion_model_key} / {champion_context_key}")
    emb_path = os.path.join(EMBEDDINGS_FOLDER, champion_model_key, champion_context_key, "embeddings.pkl")
    id_path = os.path.join(EMBEDDINGS_FOLDER, champion_model_key, champion_context_key, "passage_ids.json")
    try:
        with open(emb_path, "rb") as f:
            passage_embeddings = pickle.load(f)
        with open(id_path, "r") as f:
            passage_ids = json.load(f)
        embeddings_tensor = torch.tensor(passage_embeddings).to(device)
        print("Champion embeddings loaded successfully.")
    except FileNotFoundError:
        print(f"FATAL ERROR: Champion embeddings not found at {emb_path}. Skipping this champion.")
        continue

    # --- Load Champion Query Encoder ---
    print(f"Loading champion query encoder: {champion_model_path}")
    query_encoder = SentenceTransformer(champion_model_path, device=device)

    print(f"\nStarting enriched graph re-ranking experiment with {len(hyperparameter_combinations)} combinations for {champion_name}.")

    # --- Pre-calculate initial retrievals for all queries to avoid re-computing ---
    print("Pre-calculating initial hybrid retrievals for all queries...")
    initial_retrievals = {}
    for q in tqdm(test_data, desc=f"Initial Retrieval for {champion_name}"):
        qid = q["QuestionID"]
        query = q["Question"]
        # Dense retrieval
        query_emb = query_encoder.encode(query, convert_to_tensor=True, device=device)
        cos_scores = util.pytorch_cos_sim(query_emb, embeddings_tensor)[0]
        top_dense = torch.topk(cos_scores, k=min(K_INITIAL, len(passage_ids)))
        dense_uids = [passage_ids[idx] for idx in top_dense.indices]
        uid_to_initial_score = {passage_ids[idx]: score.item() for idx, score in zip(top_dense.indices, top_dense.values)}
        # Lexical retrieval
        bm25_scores = bm25.get_scores(query.split())
        top_bm25 = np.argsort(bm25_scores)[::-1][:K_INITIAL]
        bm25_uids = [all_passage_uids[i] for i in top_bm25]
        # Fusion
        fused_results = reciprocal_rank_fusion([dense_uids, bm25_uids])
        initial_retrievals[qid] = (fused_results, uid_to_initial_score)

    # --- Loop through hyperparameter combinations ---
    for params in tqdm(hyperparameter_combinations, desc=f"Evaluating Hyperparameters for {champion_name}"):
        parent_b, cites_b, cited_by_b, sibling_b, pagerank_w, isolation_p = params

        graph_reranked_run = {}
        for q in test_data:
            qid = q["QuestionID"]
            fused_results, uid_to_initial_score = initial_retrievals[qid]

            # Build initial candidate list
            initial_candidates = []
            for uid, rrf_score in fused_results:
                initial_candidates.append({
                    "internal_uid": uid,
                    "initial_score": uid_to_initial_score.get(uid, 0),
                    "graph_bonus": 0.0
                })

            retrieved_uids_set = {cand["internal_uid"] for cand in initial_candidates[:K_INITIAL]}

            # --- Stage 2: Enriched Graph-Based Re-ranking ---
            candidates_to_rerank = initial_candidates[:K_INITIAL]

            # Find parents for all candidates once
            uid_to_parents = {
                cand["internal_uid"]: [p for p in G.predecessors(cand["internal_uid"]) if G.get_edge_data(p, cand["internal_uid"], {}).get("type") == "PARENT_OF"]
                for cand in candidates_to_rerank
            }

            for i, cand in enumerate(candidates_to_rerank):
                current_uid = cand["internal_uid"]
                bonus_applied = False

                # Parent Bonus
                if cand["internal_uid"] in uid_to_parents and uid_to_parents[cand["internal_uid"]]:
                    parent_uid = uid_to_parents[cand["internal_uid"]][0]
                    if parent_uid in retrieved_uids_set:
                        cand["graph_bonus"] += parent_b
                        bonus_applied = True

                # Sibling Bonus
                if cand["internal_uid"] in uid_to_parents and uid_to_parents[cand["internal_uid"]]:
                    parent_uid = uid_to_parents[cand["internal_uid"]][0]
                    for j, other_cand in enumerate(candidates_to_rerank):
                        if i != j and other_cand["internal_uid"] in uid_to_parents and uid_to_parents[other_cand["internal_uid"]]:
                            if uid_to_parents[other_cand["internal_uid"]][0] == parent_uid:
                                cand["graph_bonus"] += sibling_b
                                bonus_applied = True
                                break # Only apply once per sibling group check

                # Cites Bonus (outgoing)
                for neighbor_uid in G.successors(current_uid):
                    if G.get_edge_data(current_uid, neighbor_uid, {}).get("type") == "CITES" and neighbor_uid in retrieved_uids_set:
                        cand["graph_bonus"] += cites_b
                        bonus_applied = True

                # Cited By Bonus (incoming)
                for neighbor_uid in G.predecessors(current_uid):
                    if G.get_edge_data(neighbor_uid, current_uid, {}).get("type") == "CITED_BY" and neighbor_uid in retrieved_uids_set:
                        cand["graph_bonus"] += cited_by_b
                        bonus_applied = True

                # Isolation Penalty
                if not bonus_applied:
                    cand["graph_bonus"] += isolation_p

            # Calculate final graph score and re-sort
            for cand in candidates_to_rerank:
                pagerank_bonus = pagerank_scores.get(cand["internal_uid"], 0.0) * pagerank_w
                cand["graph_score"] = cand["initial_score"] + cand["graph_bonus"] + pagerank_bonus

            graph_reranked = sorted(candidates_to_rerank, key=lambda x: x["graph_score"], reverse=True)

            # Format for evaluation
            graph_reranked_run[qid] = {}
            for cand in graph_reranked[:K_FINAL]:
                node = G.nodes[cand["internal_uid"]]
                doc_id = node.get("document_id", "")
                passage_id = node.get("passage_id", "")
                if doc_id and passage_id:
                    combined_uid = f"{doc_id}|||{passage_id}"
                    graph_reranked_run[qid][combined_uid] = float(cand["graph_score"])

        # Evaluate and store results for this hyperparameter combination
        graph_metrics = evaluate_run(graph_reranked_run, qrel)
        all_results.append({
            "champion_retriever": champion_name,
            "parent_bonus": parent_b,
            "cites_bonus": cites_b,
            "cited_by_bonus": cited_by_b,
            "sibling_bonus": sibling_b,
            "pagerank_weight": pagerank_w,
            "isolation_penalty": isolation_p,
            "Recall@10": graph_metrics["recall_10"],
            "MAP@10": graph_metrics["map_cut_10"],
            "nDCG@10": graph_metrics["ndcg_cut_10"]
        })

# --- Save and Display Final Results ---
results_df = pd.DataFrame(all_results)
# Sort by the primary metric to easily see the best combinations
results_df = results_df.sort_values(by="nDCG@10", ascending=False)

print("\n📊 Final Evaluation Results:")
print(results_df.to_string(index=False, float_format="%.4f"))

results_df.to_csv(RESULTS_OUTPUT_PATH, index=False)
print(f"\n✅ Results saved to: {RESULTS_OUTPUT_PATH}")

print("\n--- 🏆 Best Performing Configuration ---")
print(results_df.iloc[0])


Using device: cuda
Loading static components...
Pre-calculating PageRank for all graph nodes...
PageRank calculation complete.
Loaded 447 test questions.
Loaded QRELs for 447 queries.
Preparing BM25 index...
BM25 ready.

--- TESTING CHAMPION RETRIEVER: e5-large-parent_child ---
Loading champion embeddings: e5-large / parent_child
Champion embeddings loaded successfully.
Loading champion query encoder: intfloat/e5-large-v2

Starting enriched graph re-ranking experiment with 144 combinations for e5-large-parent_child.
Pre-calculating initial hybrid retrievals for all queries...


Initial Retrieval for e5-large-parent_child:   0%|          | 0/447 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Initial Retrieval for e5-large-parent_child: 100%|██████████| 447/447 [01:07<00:00,  6.65it/s]
Evaluating Hyperparameters for e5-large-parent_child: 100%|██████████| 144/144 [03:37<00:00,  1.51s/it]



--- TESTING CHAMPION RETRIEVER: e5-large-full_neighborhood ---
Loading champion embeddings: e5-large / full_neighborhood
Champion embeddings loaded successfully.
Loading champion query encoder: intfloat/e5-large-v2

Starting enriched graph re-ranking experiment with 144 combinations for e5-large-full_neighborhood.
Pre-calculating initial hybrid retrievals for all queries...


Initial Retrieval for e5-large-full_neighborhood: 100%|██████████| 447/447 [00:59<00:00,  7.45it/s]
Evaluating Hyperparameters for e5-large-full_neighborhood: 100%|██████████| 144/144 [03:32<00:00,  1.48s/it]


📊 Final Evaluation Results:
        champion_retriever  parent_bonus  cites_bonus  cited_by_bonus  sibling_bonus  pagerank_weight  isolation_penalty  Recall@10  MAP@10  nDCG@10
e5-large-full_neighborhood        0.0100       0.0100          0.0400         0.0050           0.0000             0.0000     0.3720  0.2628   0.3400
e5-large-full_neighborhood        0.0100       0.0100          0.0200         0.0050           0.1000             0.0000     0.3720  0.2628   0.3400
e5-large-full_neighborhood        0.0100       0.0100          0.0200         0.0050           0.2000             0.0000     0.3720  0.2628   0.3400
e5-large-full_neighborhood        0.0100       0.0200          0.0200         0.0050           0.1000             0.0000     0.3720  0.2628   0.3400
e5-large-full_neighborhood        0.0100       0.0200          0.0400         0.0050           0.0000             0.0000     0.3720  0.2628   0.3400
e5-large-full_neighborhood        0.0100       0.0200          0.0400        

In [ ]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  #display(Audio(url='https://www.myinstants.com/media/sounds/anime-wow-sound-effect.mp3', autoplay=True))
  display(Audio(url='https://www.myinstants.com/media/sounds/money-soundfx.mp3', autoplay=True))
## Insert whatever audio file you want above

allDone()